In [6]:
import pandas as pd
from pandas import IndexSlice as idx

In [11]:
df_nace = pd.read_csv("../data/tulli/international_trade_country_NACE_breakdown.csv", index_col=[0,1,2,3])
df_reg = pd.read_csv("../data/tulli/tulli_international_trade_region_economy-sector_breakdown.csv", encoding="iso-8859-1", index_col=[0,1,2], skiprows=1)
df_empl = pd.read_csv("../data/Regional_economic_and_environmental_data/Employment_MKregion_breakdown.csv", encoding="iso-8859-1", index_col=[0,1,2,3])

In [12]:
# Normalize df_empl using Total rows
totals = df_empl.loc[idx[:, :, "Total", :], :].groupby(["Area", "Industry", "Transaction"]).sum()
totals.index = totals.index.droplevel(1)
df_empl_scaled = df_empl.div(totals, axis=0)
df_empl_scaled.drop("Total", level=2, inplace=True)
df_empl_scaled = df_empl_scaled.groupby(["Area", "Industry"]).sum()
df_empl_scaled

2015 Original series  \
Area         Industry                                                                   
MK01 Uusimaa A Agriculture, forestry and fishing (01-03)                     0.007496   
             B Mining and quarrying (05-09)                                  0.000438   
             D, E Electricity, gas, steam and air conditioni...              0.008965   
             F Construction (41-43)                                          0.071511   
             G Wholesale and retail trade; repair of motor v...              0.133462   
...                                                                               ...   
MK21 Åland   O Public administration and defence; compulsory...              0.084819   
             P Education (85)                                                0.060185   
             Q Human health and social work activities (86-88)               0.143798   
             R, S Other service activities (90-96)                           0.050694   
             T Activities of households as employers; undiff...              0.001372   

                                                                 2016 Original series  \
Area         Industry                                                                   
MK01 Uusimaa A Agriculture, forestry and fishing (01-03)                     0.007267   
             B Mining and quarrying (05-09)                                  0.000408   
             D, E Electricity, gas, steam and air conditioni...              0.010167   
             F Construction (41-43)                                          0.073665   
             G Wholesale and retail trade; repair of motor v...              0.129209   
...                                                                               ...   
MK21 Åland   O Public administration and defence; compulsory...              0.081480   
             P Education (85)                                                0.058698   
             Q Human health and social work activities (86-88)               0.142195   
             R, S Other service activities (90-96)                           0.051668   
             T Activities of households as employers; undiff...              0.001581   

                                                                 2017 Original series  \
Area         Industry                                                                   
MK01 Uusimaa A Agriculture, forestry and fishing (01-03)                     0.006338   
             B Mining and quarrying (05-09)                                  0.000391   
             D, E Electricity, gas, steam and air conditioni...              0.010111   
             F Construction (41-43)                                          0.076904   
             G Wholesale and retail trade; repair of motor v...              0.125765   
...                                                                               ...   
MK21 Åland   O Public administration and defence; compulsory...              0.081736   
             P Education (85)                                                0.060227   
             Q Human health and social work activities (86-88)               0.145339   
             R, S Other service activities (90-96)                           0.048682   
             T Activities of households as employers; undiff...              0.001634   

                                                                 2018 Original series  \
Area         Industry                                                                   
MK01 Uusimaa A Agriculture, forestry and fishing (01-03)                     0.006139   
             B Mining and quarrying (05-09)                                  0.000392   
             D, E Electricity, gas, steam and air conditioni...              0.010020   
             F Construction (41-43)                                          0.077728   
             G Wholesale and retail trade; repair of moto

In [13]:
# df_reg has 21 unique region values, df_empl has 19
# Remove Total and unknown region values from df_reg
df_reg.drop("21 Total", level=1, inplace=True)
df_reg_unknown = df_reg.loc[(slice(None), ["20 Unknown"], slice(None)), :] # might be useful later
df_reg.drop("20 Unknown", level=1, inplace=True)

# make df_reg have the same regions as df_empl
regs_reg = df_reg.index.get_level_values(1).unique().to_list()
regs_empl = df_empl_scaled.index.get_level_values(0).unique().to_list()
mapping = {regs_reg[i] : regs_empl[i] for i in range(19)}
index = df_reg.index.to_list()
index = [(ind[0], mapping[ind[1]], ind[2]) for ind in index]
index = pd.MultiIndex.from_tuples(index, names=["Time", "Area", "Direction"])
df_reg.index = index

In [14]:
# Pick last month of each year from regional trade data
idx = pd.IndexSlice
df_reg_year_aligned = df_reg.loc[idx[[i for i in range(201512, 202112, 100)], :, :], :]

In [15]:
# Move the time dimension to the index
flattened = df_empl_scaled.to_numpy().flatten()
ind = df_empl_scaled.index.to_list()
new_ind = []
for i in ind:
    for d in [201512, 201612, 201712, 201812, 201912, 202012]:
        new_ind.append((i[0], i[1], d))
df_empl_scaled_new = pd.DataFrame(flattened, index=pd.MultiIndex.from_tuples(new_ind, names=["Area", "Industry", "Time"]), columns=["Employment"])
df_empl_scaled_new = df_empl_scaled_new.reorder_levels(["Time", "Area", "Industry"]).sort_index()

In [16]:
# Combine df_empl_scaled_new and df_reg_year_aligned into one dataframe and save it
comb = df_empl_scaled_new.join(df_reg_year_aligned["Cum. statistical value (euro) from the beginning of the year"], how="inner")
comb["Trade value"] = comb["Employment"] * comb["Cum. statistical value (euro) from the beginning of the year"]
comb.drop(["Employment", "Cum. statistical value (euro) from the beginning of the year"], axis=1, inplace=True)

In [21]:
# Save combined table and employment breakdown
comb.to_csv("../data/combined/trade_by_region_industry_direction_breakdown(employment).csv")
df_empl_scaled_new.to_csv("../data/region/employment_by_industry_distribution_per_region.csv")

In [17]:
comb

Trade value
Time   Area         Industry                                           Direction                                        
201512 MK01 Uusimaa A Agriculture, forestry and fishing (01-03)        Imports by countries of origin       2.509054e+08
                                                                       Exports by countries of destination  1.267644e+08
                    B Mining and quarrying (05-09)                     Imports by countries of origin       1.467465e+07
                                                                       Exports by countries of destination  7.414043e+06
                    D, E Electricity, gas, steam and air conditioni... Imports by countries of origin       3.000927e+08
...                                                                                                                  ...
202012 MK21 Åland   Q Human health and social work activities (86-88)  Exports by countries of destination  1.273570e+07
                    R, S Other service activities (90-96)              Imports by countries of origin       1.576792e+07
                                                                       Exports by countries of destination  4.337452e+06
                    T Activities of households as employers; undiff... Imports by countries of origin       6.477630e+05
                                                                       Exports by countries of destination  1.781872e+05

[3648 rows x 1 columns]

In [19]:
df_empl_scaled_new

Employment
Time   Area         Industry                                                      
201512 MK01 Uusimaa A Agriculture, forestry and fishing (01-03)           0.007496
                    B Mining and quarrying (05-09)                        0.000438
                    D, E Electricity, gas, steam and air conditioni...    0.008965
                    F Construction (41-43)                                0.071511
                    G Wholesale and retail trade; repair of motor v...    0.133462
...                                                                            ...
202012 MK21 Åland   O Public administration and defence; compulsory...    0.079495
                    P Education (85)                                      0.057428
                    Q Human health and social work activities (86-88)     0.149832
                    R, S Other service activities (90-96)                 0.051029
                    T Activities of households as employers; undiff...    0.002096

[1824 rows x 1 columns]